In [1]:
import torch
import numpy as np

import _game
import _bot_b

import importlib
importlib.reload(_game)
importlib.reload(_bot_b)

from _game import Game, Game_Hokom, Game_Sun, Bot_Random
from _bot_b import Bot, Model


In [ ]:
model = Model()

In [162]:
bots = [Bot(models=models) for i in range(4)]

In [163]:
game = Game_Hokom(verbose=1)
game.register(bots=bots)

In [164]:
game.hokum

0

In [130]:
game.oneround()

Bot 3 played ♦9 (♠K ♥9 ♥J ♥K ♣9 ♣J ♣K)
Bot 0 played ♦10 (♠7 ♠J ♠A ♣7 ♣Q ♦Q ♦A)
Bot 1 played ♦K (♠9 ♠Q ♥7 ♥Q ♥A ♣10 ♦7)
Bot 2 played ♦J (♠8 ♠10 ♥8 ♥10 ♣8 ♣A ♦8)
Round:1 score:16 Scores:[ 16 -16]


In [165]:
game.whole_game()

hokum: ♠
Bot 3 played ♦8 (♠7 ♠8 ♥8 ♥10 ♥A ♣Q ♦A)
Bot 0 played ♦J (♠9 ♠A ♥Q ♥K ♣10 ♦Q ♦K)
Bot 1 played ♦7 (♠10 ♠K ♥9 ♥J ♣8 ♣J ♦9)
Bot 2 played ♦10 (♠J ♠Q ♥7 ♣7 ♣9 ♣K ♣A)
Round:1 score:12 Scores:12
Bot 2 played ♣9 (♠J ♠Q ♥7 ♣7 ♣K ♣A)
Bot 3 played ♣Q (♠7 ♠8 ♥8 ♥10 ♥A ♦A)
Bot 0 played ♣10 (♠9 ♠A ♥Q ♥K ♦Q ♦K)
Bot 1 played ♣J (♠10 ♠K ♥9 ♥J ♣8 ♦9)
Round:2 score:15 Scores:27
Bot 0 played ♠9 (♠A ♥Q ♥K ♦Q ♦K)
Bot 1 played ♠10 (♠K ♥9 ♥J ♣8 ♦9)
Bot 2 played ♠Q (♠J ♥7 ♣7 ♣K ♣A)
Bot 3 played ♠8 (♠7 ♥8 ♥10 ♥A ♦A)
Round:3 score:27 Scores:54
Bot 0 played ♦K (♠A ♥Q ♥K ♦Q)
Bot 1 played ♦9 (♠K ♥9 ♥J ♣8)
Bot 2 played ♠J (♥7 ♣7 ♣K ♣A)
Bot 3 played ♦A (♠7 ♥8 ♥10 ♥A)
Round:4 score:35 Scores:89
Bot 2 played ♣7 (♥7 ♣K ♣A)
Bot 3 played ♠7 (♥8 ♥10 ♥A)
Bot 0 played ♠A (♥Q ♥K ♦Q)
Bot 1 played ♣8 (♠K ♥9 ♥J)
Round:5 score:11 Scores:100
Bot 0 played ♥K (♥Q ♦Q)
Bot 1 played ♥J (♠K ♥9)
Bot 2 played ♥7 (♣K ♣A)
Bot 3 played ♥A (♥8 ♥10)
Round:6 score:17 Scores:83
Bot 3 played ♥10 (♥8)
Bot 0 played ♥Q (♦Q)
Bot 1 played ♥9 (

In [168]:
game.bots[0].data[0]["x"].shape

torch.Size([1, 224])

In [297]:
model = Model(0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [298]:
model2 = Model(0)
model2.load_state_dict(model.state_dict())

<All keys matched successfully>

In [299]:
print(model.state_dict()['lstm.weight_ih_l0'][0,0:5])
print(model2.state_dict()['lstm.weight_ih_l0'][0,0:5])

tensor([ 0.0762,  0.0847,  0.0408, -0.0605, -0.0774])
tensor([ 0.0762,  0.0847,  0.0408, -0.0605, -0.0774])


In [300]:
log_probs = []
rs = []
for _ in range(100):
    zs = torch.randn(1, 6, 224)
    xs = torch.randn(1, 192)
    logits = model2(zs, xs)
    dist = torch.distributions.Categorical(logits=logits)
    acts = dist.sample()
    log_prob = dist.log_prob(acts)
    log_probs.append(log_prob.detach())
    r = torch.randn(1)[0] * 10
    rs.append(r)

losses = []
for log_prob, r in zip(log_probs, rs):
    losses.append(-log_prob * r)

optimizer.zero_grad()
loss_sum = torch.stack(losses).sum()
loss_sum.backward()
optimizer.step()    

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [294]:
print(model.state_dict()['lstm.weight_ih_l0'][0,0:5])
print(model2.state_dict()['lstm.weight_ih_l0'][0,0:5])

tensor([ 0.0405, -0.0289, -0.0490, -0.0460, -0.0867])
tensor([ 0.0405, -0.0289, -0.0490, -0.0460, -0.0867])


In [316]:
def eval(models):
    score = 0
    bots = [Bot(models=models), Bot_Random(), Bot(models=models), Bot_Random()]
    game1 = Game()
    game1.register_bots(bots)
    game1.whole_game()
    scores1 = game1.scores
    score = score + np.sum(game1.scores)
    
    
    bots = [Bot_Random(), Bot(models=models), Bot_Random(), Bot(models=models)]
    game2 = Game()
    game2.register_bots(bots)
    game2.whole_game()
    scores2 = game2.scores
    score = score - np.sum(game2.scores)
    
    return score, scores1, scores2

In [317]:
score, scores1, scores2 = eval(models)
score

np.int64(162)

In [321]:
np.abs(scores2).sum()

np.int64(152)